[**Connect 4**](https://www.kaggle.com/competitions/connect-4)

[**Connect X**](https://www.kaggle.com/competitions/connectx/leaderboard)


[debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)


[agent_ConnectX - Marc Paulo](https://www.kaggle.com/code/marcpaulo/numpy-n-stepsla)

[agent_ConnectX - David Abelin](https://www.kaggle.com/davidabelin)

[agent_ConnectX - Indraneel Dey](https://www.kaggle.com/code/indraneeldey/minimax-optimized-with-alpha-beta-pruning)

[agent_ConnectX - Marco De Paolini](https://www.kaggle.com/code/marcodepaolini/faster-alpha-beta-pruning-n-step-lookahead)

[agent_ConnectX - KaggleLearn - Alexis Cook](https://www.kaggle.com/code/alexisbcook/n-step-lookahead)

## Hyperparameters

In [1]:
N_STEPS__MarcPaulo               = 3
N_STEPS__DavidAbelin             = [3,4]
N_STEPS__Indraneel_Dey           = 3
N_STEPS__Marco_De_Paolini        = [4,5,6]
N_STEPS__Kaggle_Learn_AlexisCook = 3

## [agent_ConnectX - Marc Paulo](https://www.kaggle.com/code/marcpaulo/numpy-n-stepsla)

In [2]:
def agent_ConnectX_MarcPaulo(obs, config) -> int:

    import numpy as np
    
    ########################################
    ###  Utilities and Helper functions  ###
    ########################################
    
    def _drop_piece(board: np.ndarray, column: int, mark: int) -> np.ndarray:
        """
        Play one turn and return the next board.
        Drop the given <mark> on the given <column>
        The given action (<column>) must be legal.

        :param board: game board
        :param column: action
        :param mark: player
        :return: next board after the turn ('column') is played
        """

        is_illegal_action = board[0, column] in np.where(board[0] != 0)[0]
        if is_illegal_action:
            raise Exception(
                "illegal action! can't drop piece", f"{board}; {column}"
            )
        next_board = board.copy()
        row = np.where(board[:, column] == 0)[0][-1]
        next_board[row, column] = mark
        return next_board
    
    
    def _count_n_in_row(
        board: np.ndarray, 
        n: int, 
        mark: int, 
        inrow: int = 4
    ) -> int:
        """
        Returns the number of occurrences of <n> <marks> in row, with the
        possibility of completing <inrow> pieces in that line 
        (i.e. the opponent is not blocking the line).

        :param board: game board
        :param n: number of <marks> in a row
        :param mark: player's marks
        :param inrow: how many marks in a row are required to win the game
        :return: number of occurrences of the given pattern
        """

        count = 0
        # functions to check that there are <n> <marks> and <n>-1 empty positions
        check_marks = lambda ww: (ww == mark).sum(axis=1) == n
        check_spaces = lambda ww: (ww == 0).sum(axis=1) == inrow-n
        # check rows
        for j in range(board.shape[1] - inrow + 1):
            w = board[:, j:j+inrow]  # window
            count += np.sum(check_spaces(w) & check_marks(w))
        # check columns
        for i in range(board.shape[0] - inrow + 1):
            w = board[i:i+inrow, :].T  # window
            count += np.sum(check_spaces(w) & check_marks(w))
        # check diagonals
        check_marks = lambda ww: (ww == mark).sum() == n
        check_spaces = lambda ww: (ww == 0).sum() == inrow-n
        for i in range(board.shape[0] - inrow + 1):
            for j in range(board.shape[1] - inrow + 1):
                asc_diag = board[i:i+inrow, j:j+inrow].diagonal()
                des_diag = np.fliplr(board[i:i+inrow, j:j+inrow]).diagonal()
                count += np.sum(check_spaces(asc_diag) & check_marks(asc_diag))
                count += np.sum(check_spaces(des_diag) & check_marks(des_diag))
        return count
    
    
    def _is_game_winner(board: np.ndarray, mark: int, inrow: int = 4) -> bool:
        """
        Checks if the given player (<mark>) is
        the winner of the game (<board>).

        :param board: connectX game board
        :param mark: player
        :param inrow: how many pieces in row in order to win
        :return: True if 'mark' is the winner of the game
        """

        target = np.array([mark] * inrow)  # winning combination
        # check rows
        for j in range(board.shape[1] - inrow + 1):
            # look at n-length horizontal sequences starting at column 'j'
            found = np.any(np.all(target == board[:, j:j+inrow], axis=1))
            if found:
                return True
        # check columns
        for i in range(board.shape[0] - inrow + 1):
            # look at n-length vertical sequences starting at row 'i'
            found = np.any(np.all(target == board[i:i+inrow, :].T, axis=1))
            if found:
                return True
        # check diagonals (ascending and descending)
        for i in range(board.shape[0] - inrow + 1):
            for j in range(board.shape[1] - inrow + 1):
                asc_diag = board[i:i+inrow, j:j+inrow].diagonal()
                des_diag = np.fliplr(board[i:i+inrow, j:j+inrow]).diagonal()
                if np.all(asc_diag == target) or np.all(des_diag == target):
                    return True
        return False
    
    
    def _is_terminal(board: np.ndarray, inrow: int = 4) -> bool:
        """
        Checks if the given state (<board>)
        is a terminal state (game over)

        :param board: game board
        :param inrow: how many tokens in line to win the game
        :return:
        """

        return (board == 0).sum() == 0 or \
            _is_game_winner(board=board, mark=1, inrow=inrow) or \
            _is_game_winner(board=board, mark=2, inrow=inrow)
    
    
    def _score_leaf_board(board: np.array) -> float:
        """
        Computes and returns the score of the given "leaf" board.
        The higher the score, the more valuable the board is.
        Count how many times each pattern appears in the board, and use this
        counter to weight their pattern_score.

        :param board: game board
        :return: score assigned to the given observation
        """
    
        # Define the patterns to look for and their scores
        _pattern_scores = {
            4: 1e10,  # four of your tokens in a row
            3: 1e4,   # three of your tokens in a row
            2: 1e2,   # two of your tokens in a row
           -2: -1,    # two of the opponent's tokens in a row
           -3: -1e6,  # three of the opponent's tokens in a row
           -4: -1e8   # four of the opponent's tokens in a row
        }
        # the score is a weighted sum (counts) of the _pattern_scores
        score = 0
        for pattern, pattern_score in _pattern_scores.items():
            mark = 1 if pattern > 0 else -1
            counts = _count_n_in_row(board=board, n=abs(pattern), mark=mark)
            score += counts * pattern_score
        return score
    
    
    def _minmax_search(
        board: np.ndarray,
        depth: int,
        is_max_player: bool
    ) -> float:
        """
        Runs a recursive min-max search. It goes N-1 steps down the game tree,
        starting at <board>. It scores the leaf nodes, and then goes up applying
        the minimax search to minimize the possible loss for a worst case scenario.

        :param board: game board
        :param depth: depth to grow the search tree
        :param is_max_player: whether player aims to minimize of maximize score
        :return: score of the given observation (board, mark)
        """

        # Base Case:
        if depth == 0 or _is_terminal(board=board):
            return _score_leaf_board(board=board)
        # Recursive Steps:
        legal_actions = np.where(board[0] == 0)[0]
        if is_max_player:
            value = - np.Inf
            for action in legal_actions:
                child_board = _drop_piece(board=board, column=action, mark=1)
                value = max(
                    value, 
                    _minmax_search(board=child_board, depth=depth-1, is_max_player=False)
                )
            return value
        else:  # min-player
            value = np.Inf
            for action in legal_actions:
                child_board = _drop_piece(board=board, column=action, mark=-1)
                value = min(
                    value, 
                    _minmax_search(board=child_board, depth=depth-1, is_max_player=True)
                )
            return value
        
    
    def _compute_scores(board: np.ndarray, n: int) -> np.array:
        """
        Given an observation, go <n> steps down the game tree, score the
        leaf nodes, and then go up applying the minimax search. Return
        the scores of the available actions in the current turn.

        :param board: game board
        :param n: depth of the MinMax Search
        :return: scores of the available actions in the current turn
        """

        scores = np.full(board.shape[1], -np.Inf)
        legal_actions = np.where(board[0] == 0)[0]
        # illegal actions will receive a -inf score and won't be chosen
        for action in legal_actions:
            next_board = _drop_piece(board=board, column=action, mark=1)
            scores[action] = _minmax_search(
                board=next_board, depth=n-1, is_max_player=False
            )
        return scores
    
    
    # begin to work..
    
    try:
        # From list to numpy array with the right shape
        board_ = np.array(obs.board).reshape(config.rows, config.columns)
        # New encoding = {0:empty, 1:active_player, -1:oppponent}
        opponent = 1 if obs.mark == 2 else 2
        board_[board_ == opponent] = -1
        board_[board_ == obs.mark] = 1
        
        # Select action:
        N_STEPS = 3 # N_STEPS__MarcPaulo
        
        scores = _compute_scores(board=board_, n = N_STEPS)
        best_scored_actions = np.where(scores == np.amax(scores))[0]
        best_central_action = best_scored_actions[
            np.argmin(abs(best_scored_actions - config.columns//2))
        ]
        return int(best_central_action)
    
    except:  # just in case something goes wrong
        # Select the most central column (among the legal columns)
        legal_actions = np.where(board_[0] == 0)[0]
        legal_central_action = legal_actions[
            np.argmin(abs(legal_actions - config.columns//2))
        ]
        return int(legal_central_action)


## [agent_ConnectX - David Abelin](https://www.kaggle.com/code/davidabelin/alphabeta-agent)

In [3]:
#@title Standard Heuristic Agent with Adjustable Depth
def agent_ConnectX_DavidAbelin(obs, config):
    # config is dict: {'rows': 6, 'columns': 7, 'inarow': 4}
    # obs.board is last move of opponent, obs.mark is current player
    # returns the column that max's next grid's score

    import numpy as np
    import random

    # constants
    ROWS    = config.rows
    COLUMNS = config.columns
    CNCTX   = config.inarow
    
    # vary lookahead depth according to state of play:  N_STEPS__DavidAbelin = [3,4]
    
    if obs.board.count(0) >= ROWS*COLUMNS/2:
        N_STEPS = 3 # N_STEPS__DavidAbelin [0]
    else:
        N_STEPS = 4 # N_STEPS__DavidAbelin [1] # deeper search after half the board is filled
        
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece):
        return (window.count(piece) == num_discs and window.count(0) == CNCTX-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece):
        num_windows = 0
        # horizontal
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if check_window(window, num_discs, piece):
                    num_windows += 1
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if check_window(window, num_discs, piece):
                    num_windows += 1
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    num_windows += 1
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    num_windows += 1
        return num_windows

    # Helper function for minimax: calculates value of heuristic for grid, ! original heuristic function
    def get_heuristic(grid, mark):  
        
        # heuristic coefficients
        
        A              =    1  #  my - twos
        B              =   50  #  my - threes
        C              =  500  #  my - fours    
        
        D              =  -10  # opp - threes
        E              = -100  # opp - fours

        num_twos       = count_windows(grid, 2, mark)     # A
        num_threes     = count_windows(grid, 3, mark)     # B
        num_fours      = count_windows(grid, 4, mark)     # C
        
        num_threes_opp = count_windows(grid, 3, mark%2+1) # D
        num_fours__opp = count_windows(grid, 4, mark%2+1) # E
        
        score = A*num_twos + B*num_threes + C*num_fours + D*num_threes_opp + E*num_fours__opp
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window):
        return window.count(1) == CNCTX or window.count(2) == CNCTX

    # Helper function for minimax: checks if game has ende
    def is_terminal_node(grid):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if is_terminal_window(window):
                    return True
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if is_terminal_window(window):
                    return True
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if is_terminal_window(window):
                    return True
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if is_terminal_window(window):
                    return True
        return False

    # Minimax implementation was here:
    def minimax_ab(node, depth, alpha, beta, maximizingPlayer, mark, config):

        if depth == 0 or is_terminal_node(node):
            return get_heuristic(node, mark)
        
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]

        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark)
                value = max(value, minimax_ab(child, depth-1, alpha, beta, False, mark, config))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1)
                value = min(value, minimax_ab(child, depth-1, alpha, beta, True, mark, config))
                beta  = min(beta,value)
                if beta <= alpha:
                    break 
            return value

    # Uses minimax_ab to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark)
        score  = minimax_ab(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark, config)
        return score

    #########################
    # Agent makes selection #
    #########################

    # Get list of valid moves
    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]

    # Convert the board to a 2D grid
    ########## ENTER OBS:
    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)

    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))

    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select at random from the maximizing columns
    return random.choice(max_cols)

## [agent_ConnectX - Indraneel Dey](https://www.kaggle.com/code/indraneeldey/minimax-optimized-with-alpha-beta-pruning)

In [4]:
def agent_ConnectX_Indraneel_Dey(obs, config):
    import numpy as np

    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows - 1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid
    
    def count_windows(grid, piece, config):
        num_2, num_3, num_4, num_2_opp, num_3_opp, num_4_opp, part_2, part_2_opp, avail_2, avail_3, avail_2_opp, avail_3_opp = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        for row in range(config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = np.array(list(grid[row, col: col + config.inarow]))
                num_piece = np.count_nonzero(window == piece)
                num_opp = np.count_nonzero(window == 3 - piece)
                num_0 = np.count_nonzero(window == 0)
                if num_piece == config.inarow:
                    num_4 += 1
                elif num_opp == config.inarow:
                    num_4_opp += 1
                elif num_piece == config.inarow - 1 and num_0 == 1:
                    if row == config.rows - 1:
                        avail_3 += 1
                    else:
                        if grid[row + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3 += 1
                        else:
                            avail_3 += 1
                elif num_opp == config.inarow - 1 and num_0 == 1:
                    if row == config.rows - 1:
                        avail_3_opp += 1
                    else:
                        if grid[row + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3_opp += 1
                        else:
                            avail_3_opp += 1
                elif num_piece == config.inarow - 2 and num_0 == 2:
                    if row == config.rows - 1:
                        avail_2 += 1
                    else:
                        count = 0
                        for index in np.where(window == 0)[0]:
                            if grid[row + 1, col + index] != 0:
                                count += 1
                        if count == 2:
                            avail_2 += 1
                        elif count == 1:
                            part_2 += 1
                        else:
                            num_2 += 1
                elif num_opp == config.inarow - 2 and num_0 == 2:
                    if row == config.rows - 1:
                        avail_2_opp += 1
                    else:
                        count = 0
                        for index in np.where(window == 0)[0]:
                            if grid[row + 1, col + index] != 0:
                                count += 1
                        if count == 2:
                            avail_2_opp += 1
                        elif count == 1:
                            part_2_opp += 1
                        else:
                            num_2_opp += 1
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns):
                window = list(grid[row: row + config.inarow, col])
                num_piece = window.count(piece)
                num_opp = window.count(3 - piece)
                num_0 = window.count(0)
                if num_piece == config.inarow:
                    num_4 += 1
                elif num_opp == config.inarow:
                    num_4_opp += 1
                elif num_piece == config.inarow - 1 and num_0 == 1:
                    avail_3 += 1
                elif num_opp == config.inarow - 1 and num_0 == 1:
                    avail_3_opp += 1
                elif num_piece == config.inarow - 2 and num_0 == 2:
                    part_2 += 1
                elif num_opp == config.inarow - 2 and num_0 == 2:
                    part_2_opp += 1
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns - (config.inarow - 1)):
                window = np.array(list(grid[range(row, row + config.inarow), range(col, col + config.inarow)]))
                num_piece = np.count_nonzero(window == piece)
                num_opp = np.count_nonzero(window == 3 - piece)
                num_0 = np.count_nonzero(window == 0)
                if num_piece == config.inarow:
                    num_4 += 1
                elif num_opp == config.inarow:
                    num_4_opp += 1
                elif num_piece == config.inarow - 1 and num_0 == 1:
                    if row + np.where(window == 0)[0][0] == config.rows - 1:
                        avail_3 += 1
                    else:
                        if grid[row + np.where(window == 0)[0][0] + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3 += 1
                        else:
                            avail_3 += 1
                elif num_opp == config.inarow - 1 and num_0 == 1:
                    if row + np.where(window == 0)[0][0] == config.rows - 1:
                        avail_3_opp += 1
                    else:
                        if grid[row + np.where(window == 0)[0][0] + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3_opp += 1
                        else:
                            avail_3_opp += 1
                elif num_piece == config.inarow - 2 and num_0 == 2:
                    count = 0
                    for index in np.where(window == 0)[0]:
                        if row + index == config.rows - 1:
                            count += 1
                        elif grid[row + index + 1, col + index] != 0:
                            count += 1
                    if count == 2:
                        avail_2 += 1
                    elif count == 1:
                        part_2 += 1
                    else:
                        num_2 += 1
                elif num_opp == config.inarow - 2 and num_0 == 2:
                    count = 0
                    for index in np.where(window == 0)[0]:
                        if row + index == config.rows - 1:
                            count += 1
                        elif grid[row + index + 1, col + index] != 0:
                            count += 1
                    if count == 2:
                        avail_2_opp += 1
                    elif count == 1:
                        part_2_opp += 1
                    else:
                        num_2_opp += 1
        for row in range(config.inarow - 1, config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = np.array(list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)]))
                num_piece = np.count_nonzero(window == piece)
                num_opp = np.count_nonzero(window == 3 - piece)
                num_0 = np.count_nonzero(window == 0)
                if num_piece == config.inarow:
                    num_4 += 1
                elif num_opp == config.inarow:
                    num_4_opp += 1
                elif num_piece == config.inarow - 1 and num_0 == 1:
                    if row - np.where(window == 0)[0][0] == config.rows - 1:
                        avail_3 += 1
                    else:
                        if grid[row - np.where(window == 0)[0][0] + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3 += 1
                        else:
                            avail_3 += 1
                elif num_opp == config.inarow - 1 and num_0 == 1:
                    if row - np.where(window == 0)[0][0] == config.rows - 1:
                        avail_3_opp += 1
                    else:
                        if grid[row - np.where(window == 0)[0][0] + 1, col + np.where(window == 0)[0][0]] == 0:
                            num_3_opp += 1
                        else:
                            avail_3_opp += 1
                elif num_piece == config.inarow - 2 and num_0 == 2:
                    count = 0
                    for index in np.where(window == 0)[0]:
                        if row - index == config.rows - 1:
                            count += 1
                        elif grid[row - index + 1, col + index] != 0:
                            count += 1
                    if count == 2:
                        avail_2 += 1
                    elif count == 1:
                        part_2 += 1
                    else:
                        num_2 += 1
                elif num_opp == config.inarow - 2 and num_0 == 2:
                    count = 0
                    for index in np.where(window == 0)[0]:
                        if row - index == config.rows - 1:
                            count += 1
                        elif grid[row - index + 1, col + index] != 0:
                            count += 1
                    if count == 2:
                        avail_2_opp += 1
                    elif count == 1:
                        part_2_opp += 1
                    else:
                        num_2_opp += 1
        return num_2, num_3, num_4, num_2_opp, num_3_opp, num_4_opp, part_2, part_2_opp, avail_2, avail_3, avail_2_opp, avail_3_opp
    
    def get_heuristic(grid, mark, config):
        num_2, num_3, num_4, num_2_opp, num_3_opp, num_4_opp, part_2, part_2_opp, avail_2, avail_3, avail_2_opp, avail_3_opp = count_windows(grid, mark, config)
        return (50000 * num_4 - 10000 * num_4_opp + 5000 * avail_3 - 1000 * avail_3_opp + 500 * avail_2 - 100 * avail_2_opp + 
                500 * num_3 - 100 * num_3_opp + 50 * part_2 - 10 * part_2_opp + 5 * num_2 - 1 * num_2_opp)
    
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        return alphabeta(next_grid, nsteps - 1, False, mark, config)
    
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow
    
    def is_terminal_node(grid, config): 
        if list(grid[0, :]).count(0) == 0:
            return True
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow - 1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns):
                window = list(grid[row: row + config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row + config.inarow), range(col, col + config.inarow)])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.inarow - 1, config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False

    def alphabeta(node, depth, maximizingPlayer, mark, config, alpha=-np.inf, beta=np.inf):
        if depth == 0 or is_terminal_node(node, config):
            return get_heuristic(node, mark, config)
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        if maximizingPlayer:
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                alpha = max(alpha, alphabeta(child, depth - 1, False, mark, config, alpha, beta))
                if alpha >= beta:
                    return beta
            return alpha
        else:
            for col in valid_moves:
                child = drop_piece(node, col, 3 - mark, config)
                beta = min(beta, alphabeta(child, depth - 1, True, mark, config, alpha, beta))
                if alpha >= beta:
                    return alpha
            return beta
    
    def check_winning_move(obs, config, col, piece):
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        for row in range(config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(next_grid[row, col: col + config.inarow])
                if window.count(piece) == config.inarow:
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns):
                window = list(next_grid[row : row + config.inarow, col])
                if window.count(piece) == config.inarow:
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(next_grid[range(row, row + config.inarow), range(col, col + config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        for row in range(config.inarow - 1, config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(next_grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        return False
    
    N_STEPS = 3 # N_STEPS__Indraneel_Dey
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    winning_moves = [move for move in valid_moves if check_winning_move(obs, config, move, obs.mark)]
    if winning_moves:
        return winning_moves[0]
    losing_moves = [move for move in valid_moves if check_winning_move(obs, config, move, 3 - obs.mark)]
    if losing_moves:
        return losing_moves[0]
    temp_moves = valid_moves.copy()
    for col in temp_moves:
        if grid[1][col] == 0:
            next_grid = drop_piece(grid, col, obs.mark, config)
            temp = obs
            temp.board = next_grid.flatten()
            if check_winning_move(temp, config, col, 3 - obs.mark):
                temp_moves.remove(col)
    if temp_moves:
        valid_moves = temp_moves
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    return min(max_cols, key=lambda x: abs(3 - x))


## [agent_ConnectX - Marco De Paolini](https://www.kaggle.com/code/marcodepaolini/faster-alpha-beta-pruning-n-step-lookahead)


In [5]:
def agent_ConnectX_Marco_De_Paolini(obs, config):
    import random
    import numpy as np
    
#     print("Step {}: AphaBeta Agent moving".format(obs.step))

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Get the number of pieces of the same mark in a window
    def pieces_in_window(window, piece):
        return window.count(piece) * (window.count(piece) + window.count(0) == config.inarow)
    
    # Counts number of pieces for both players for every possible window
    def count_windows(grid):
        windows = {piece: [0 for i in range(config.inarow+1)] for piece in [1, 2]}
        
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                windows[1][pieces_in_window(window, 1)]+=1
                windows[2][pieces_in_window(window, 2)]+=1

        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                windows[1][pieces_in_window(window, 1)]+=1
                windows[2][pieces_in_window(window, 2)]+=1

        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                windows[1][pieces_in_window(window, 1)]+=1
                windows[2][pieces_in_window(window, 2)]+=1

        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                windows[1][pieces_in_window(window, 1)]+=1
                windows[2][pieces_in_window(window, 2)]+=1
        return windows
                
    # Calculates value of heuristic for grid
    def get_heuristic(grid, mark):
        windows=count_windows(grid)
        score =  windows[mark][1] + windows[mark][2]*3 + windows[mark][3]*9 + windows[mark][4]*81 - windows[mark%2+1][1] - windows[mark%2+1][2]*3 - windows[mark%2+1][3]*9 - windows[mark%2+1][4]*81
        return score
    
    # Uses alphabeta to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, nsteps):
        next_grid = drop_piece(grid, col, mark)
        score = alphabeta(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark)
        return score

    # Checks if game has ended
    def is_terminal_node(grid):
        windows=count_windows(grid)
        return windows[1][config.inarow] + windows[2][config.inarow] > 0

    # Alpha Beta pruning implementation
    def alphabeta(node, depth, a, b, maximizingPlayer, mark):
        is_terminal = is_terminal_node(node)
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark)
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark)
                value = max(value, alphabeta(child, depth-1, a, b, False, mark))
                a = max(a, value)
                if a >= b:
                    break # β cutoff
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1)
                value = min(value, alphabeta(child, depth-1, a, b, True, mark))
                b = min(b, value)
                if b <= a:
                    break # α cutoff
            return value

    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    n_steps = 4 if obs.board.count(0)>len(obs.board)*2/3 else 5 if obs.board.count(0)>len(obs.board)/3 else 6
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, n_steps) for col in valid_moves]))
#     print("Scores:", scores, end=' - ')
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    move = random.choice(max_cols)
#     print("Selected move:", move)
    return move

## [agent_ConnectX - KaggleLearn - Alexis Cook](https://www.kaggle.com/code/alexisbcook/n-step-lookahead)

In [6]:
def agent_ConnectX_Kaggle_Learn_AlexisCook(obs, config):
    import random
    import numpy as np

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows

    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)
        score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, False, mark, config)
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False

    # Minimax implementation
    def minimax(node, depth, maximizingPlayer, mark, config):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth-1, False, mark, config))
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, minimax(child, depth-1, True, mark, config))
            return value
        
    # begin to work..
    
    # How deep to make the game tree: higher values take longer to run!
    
    N_STEPS = 3 # N_STEPS__Kaggle_Learn_AlexisCook 
    
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [7]:
from kaggle_environments import make, evaluate

env__6_7 = make("connectx",configuration={"inarow":4,"columns":7,"rows":6})
env__8_8 = make("connectx",configuration={"inarow":4,"columns":8,"rows":8})

No pygame installed, ignoring import


In [8]:
env__6_7.run([agent_ConnectX_MarcPaulo, "negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [9]:
env__6_7.run([agent_ConnectX_DavidAbelin, "negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [10]:
env__6_7.run([agent_ConnectX_Indraneel_Dey, "negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [11]:
env__6_7.run([agent_ConnectX_Marco_De_Paolini, "negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [12]:
env__6_7.run([agent_ConnectX_Kaggle_Learn_AlexisCook, "negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [13]:
env__6_7.run([agent_ConnectX_MarcPaulo, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__6_7.render(mode="ipython", width=500, height=450)

In [14]:
env__6_7.run([agent_ConnectX_DavidAbelin, agent_ConnectX_Marco_De_Paolini])
env__6_7.render(mode="ipython", width=500, height=450)

In [15]:
env__6_7.run([agent_ConnectX_Indraneel_Dey, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__6_7.render(mode="ipython", width=500, height=450)

In [16]:
env__6_7.run([agent_ConnectX_MarcPaulo, agent_ConnectX_Indraneel_Dey])
env__6_7.render(mode="ipython", width=500, height=450)

In [17]:
env__8_8.run([agent_ConnectX_MarcPaulo,"negamax"])
env__8_8.render(mode="ipython", width=550, height=500)

In [18]:
env__8_8.run([agent_ConnectX_DavidAbelin, "negamax"])
env__8_8.render(mode="ipython", width=550, height=500)

In [19]:
env__8_8.run([agent_ConnectX_Indraneel_Dey,"negamax"])
env__8_8.render(mode="ipython", width=550, height=500)

In [20]:
env__8_8.run([agent_ConnectX_Marco_De_Paolini,"negamax"])
env__8_8.render(mode="ipython", width=550, height=500)

In [21]:
env__8_8.run([agent_ConnectX_Kaggle_Learn_AlexisCook,"negamax"])
env__8_8.render(mode="ipython", width=550, height=500)

In [22]:
env__8_8.run([agent_ConnectX_MarcPaulo, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__8_8.render(mode="ipython", width=550, height=500)

In [23]:
env__8_8.run([agent_ConnectX_DavidAbelin, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__8_8.render(mode="ipython", width=550, height=500)

In [24]:
env__8_8.run([agent_ConnectX_Marco_De_Paolini, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__8_8.render(mode="ipython", width=550, height=500)

In [25]:
env__8_8.run([agent_ConnectX_Indraneel_Dey, agent_ConnectX_Kaggle_Learn_AlexisCook])
env__8_8.render(mode="ipython", width=550, height=500)

In [26]:
env__8_8.run([agent_ConnectX_MarcPaulo, agent_ConnectX_Indraneel_Dey])
env__8_8.render(mode="ipython", width=550, height=500)

In [27]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(agent_ConnectX_MarcPaulo, "agent_cx_MarcPaulo.py")
write_agent_to_file(agent_ConnectX_DavidAbelin,  "agent_cx_DavidAbelin.py")
write_agent_to_file(agent_ConnectX_Indraneel_Dey,   "agent_cx_Indraneel_Dey.py")
write_agent_to_file(agent_ConnectX_Marco_De_Paolini,   "agent_cx_Marco_De_Paolini.py")
write_agent_to_file(agent_ConnectX_Kaggle_Learn_AlexisCook,"agent_cx_Kaggle_Learn_AlexisCook.py")

<function agent_ConnectX_MarcPaulo at 0x7cedc42d31c0> written to agent_cx_MarcPaulo.py
<function agent_ConnectX_DavidAbelin at 0x7cedc42d3a30> written to agent_cx_DavidAbelin.py
<function agent_ConnectX_Indraneel_Dey at 0x7cedc42d3130> written to agent_cx_Indraneel_Dey.py
<function agent_ConnectX_Marco_De_Paolini at 0x7cedc42d3d90> written to agent_cx_Marco_De_Paolini.py
<function agent_ConnectX_Kaggle_Learn_AlexisCook at 0x7cedc42d28c0> written to agent_cx_Kaggle_Learn_AlexisCook.py


## [debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

In [28]:
def debugger_c4(list_of_agents_to_trace):
    map4q = '''
         01,02,03,04; 31,32,33,34;                 32,33,34,35; 33,34,35,36; 
           40,41,42,43; 41,42,43,44;             42,43,44,45; 43,44,45,46; 
             50,51,52,53; 51,52,53,54;         52,53,54,55; 30,31,32,33;
               04,14,24,34; 14,24,34,44;     24,34,44,54; 05,15,25,35; 
                 06,16,26,36; 03,12,21,30; 15,25,35,45; 25,35,45,55; 
                   16,26,36,46; 26,36,46,56;          23,34,45,56;
                     04,13,22,31; 13,22,31,40;      03,14,25,36; 
                       05,14,23,32; 14,23,32,41;  20,30,40,50; 
                         06,15,24,33; 15,24,33,42;
                           24,33,42,51; 23,32,41,50;
                             03,13,23,33; 13,23,33,43; 
                               16,25,34,43; 25,34,43,52; 
                                 11,22,33,44; 11,21,31,41;
                     01,12,23,34;  02,13,24,35; 12,23,34,45; 
                   00,11,22,33;      26,35,44,53; 23,33,43,53;              
                 02,12,22,32;          21,32,43,54; 22,33,44,55; 
               00,10,20,30; 22,23,24,25; 21,31,41,51; 13,24,35,46;
             01,11,21,31; 10,20,30,40;     12,22,32,42; 22,32,42,52;
           00,01,02,03; 10,21,32,43;         02,03,04,05; 03,04,05,06;
         10,11,12,13; 11,12,13,14;             12,13,14,15; 13,14,15,16;
       20,21,22,23; 21,22,23,24;                 23,24,25,26; 20,31,42,53; 53,54,55,56'''
    
    # -----------------------------------------------------------------------------------------------
    start_games_at_away_first_agent = False # True # => The first agent from the list will start his game either “from home” or “away”,
    only_observation_first_agent    = False # True # => Everyone will play two games with the first: one “at home”, the other “away”, or everyone will play with everyone
    # -----------------------------------------------------------------------------------------------
    class Obs:
        def __init__(self, board, mark):
            self.board = board
            self.mark = mark
    # -----------------------------------------------------------------------------------------------
    class Config:
        def __init__(self, rows, columns, inarow):
            self.rows = rows
            self.columns = columns
            self.inarow = inarow
    # -----------------------------------------------------------------------------------------------
    import time
    import numpy as np
    # -----------------------------------------------------------------------------------------------
    Rows, Cols = 6, 7
    config = Config(Rows, Cols, 4)
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    # -----------------------------------------------------------------------------------------------
    def yx(s): return AT2[int(s[0])][int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # -----------------------------------------------------------------------------------------------
    def gyx(s): return [int(s[0]), int(s[1])]
    gSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    gMap4q = [list(map(gyx, s)) for s in gSs]
    # -----------------------------------------------------------------------------------------------
    def inject(col, mark, board):
        i_krai = config.columns * (config.rows - 1) + col
        for i in range(i_krai, -1, -7):
            if board[i] == 0:
                new_board = board.copy()
                new_board[i] = mark
                return new_board
        return board
    # -----------------------------------------------------------------------------------------------
    def win_4o_in(board):
        grid = np.asarray(board).reshape(6,7)
        for i in gMap4q:
            v = grid[i[0][0],i[0][1]] * grid[i[1][0],i[1][1]] * grid[i[2][0],i[2][1]] * grid[i[3][0],i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # -----------------------------------------------------------------------------------------------
    def tsum(reagent):
        home = sum(reagent["games"][0]) if len(reagent["games"][0])>0 else 0
        away = sum(reagent["games"][1]) if len(reagent["games"][1])>0 else 0
        return home + away
    # -----------------------------------------------------------------------------------------------
    def ttime(reagent, gt):
        time = reagent["time"][0]  if len(reagent["time"])>0 else 0
        return [round(time+gt, 2)]
    # -----------------------------------------------------------------------------------------------
    def tmoves(reagent, ms):
        moves = reagent["moves"][0] if len(reagent["moves"])>0 else 0
        return [moves+ms]
    # -----------------------------------------------------------------------------------------------
    def tper1move(reagent):
        time = reagent["time"][0]   if len(reagent["time"]) > 0 else 0
        moves = reagent["moves"][0] if len(reagent["moves"])> 0 else 0
        return [round(time / moves, 1)]
    # -----------------------------------------------------------------------------------------------
    def print_to_clip(board, ims, i, attack, defense, stime1, stime2, s1, s2):
        print("1.{0} [{1}]   2.{2} [{3}]".format(name(attack), s1, name(defense), s2),
              "  1.Time =", round(stime1, 2), '  2.Time =', round(stime2, 2), '  q.moves:', i,
              "  1.speed =", round(stime1 / i, 1), "  2.speed =", round(stime2 / i, 1),
              "  \n\nGame:", str(ims).replace(", ", ","),
              "  \n\nBoard:", str(board).replace(", ", ","),"\n")
        bn2d = np.array(board).reshape(Rows, Cols)
        for r in range(len(bn2d)): print(bn2d[r])
        print('{0}{1}'.format('\n',"="*121))
    # -----------------------------------------------------------------------------------------------
    def rec(total_points, _A, _D, ia, id, stime1, stime2):
        reatta = total_points[name(Attack)]
        redefe = total_points[name(Defense)]
        reatta["games"][0].extend([_A])
        redefe["games"][1].extend([_D])
        reatta["T"] = [tsum(reatta)]
        redefe["T"] = [tsum(redefe)]
        reatta["time"] = ttime(reatta, stime1)
        redefe["time"] = ttime(redefe, stime2)
        reatta["moves"] = tmoves(reatta, ia)
        redefe["moves"] = tmoves(redefe, id)
        reatta["speed"] = tper1move(reatta)
        redefe["speed"] = tper1move(redefe)
    # -----------------------------------------------------------------------------------------------
    def agent_work(Agent, board, mark, config):
        obs = Obs(board, mark)
        start = time.time()
        im = Agent(obs, config)
        t = time.time() - start
        board_next = inject(im, obs.mark, board)
        return im, t, board_next
    # -----------------------------------------------------------------------------------------------
    def write_Connect4(ims):
        #     with open("Connect4.txt", "w") as file:
        #         file.write(str(ims))
        pass 
    # -----------------------------------------------------------------------------------------------
    def write_uraConnect(ura_ims, Attack, Defense):
        #     try:
        #         with open("uraConnect.txt", "w") as file:
        #             stf = str(ura_ims) + '\n' + name(Attack) + ' - ' + name(Defense)
        #             file.write(stf)
        #     finally: return
        pass
    # -----------------------------------------------------------------------------------------------
    def name(agent): return agent.__name__.replace("my_","")
    # -----------------------------------------------------------------------------------------------

    Agents = list_of_agents_to_trace # [ag1,ag2,..,agn]

    observatory = Agents[0] if only_observation_first_agent and len(Agents) > 1 else None

    total_points = { name(agent):{"T":[],"games":[[],[]],"time":[],"moves":[],"speed":[]} for agent in Agents }

    ig, board__0 = 1, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for home in Agents:
        for away in Agents:
            if home!=away and (observatory==None or observatory!=None and (observatory==home or observatory==away)):
                if ig==1:
                    print("\n")
                    print(ig-1, ":")
                    print('----------------')
                    for itp in total_points.items(): print(itp)
                    ig += 1
                stime1, stime2, imts1, imts2, ims, imt, = 0, 0, [], [], [], [[], []]
                board__1 = board__0
                # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if start_games_at_away_first_agent:
                    Attack, Defense = away, home
                else:
                    Attack, Defense = home, away
                # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                total_points_Attack, total_points_Defense = [], []
                print("--------------------------")
                print(Attack.__name__, "  vs  ", Defense.__name__)
                print("-----------------------------------")
                for i in range(1,22):
                    im1, t1, board__2  = agent_work(Attack, board__1, 1, config)
                    stime1 +=t1
                    imt[0] = str(round(t1,0))
                    ims.append(im1)
                    imts1.append(imt[0])
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__2):
                        rec(total_points, 3, 0, i, i-1, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   1 1  1   1    1     1     ', Attack.__name__, "     1     1    1   1  1 1")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip (board__2, ims, i, Attack, Defense, stime1, stime2, 3,0)
                        write_Connect4 (ims)
                        write_uraConnect (ims, Attack, Defense)
                        time.sleep(1)
                        break
                    im2, t2, board__1 = agent_work(Defense, board__2, 2, config)
                    stime2 +=t2
                    imt[1] = str(round(t2, 0))
                    ims.append(im2)
                    imts2.append(imt[1])
                    print('({0}) . {1} > {2}  {3} < {4} . ({5}) --- {6} : {7} '
                          .format(round(t1,1), name(Attack), im1, im2, name(Defense), round(t2,1), i, str(ims).replace(", ",",")))
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__1):
                        rec(total_points, 0, 3, i, i, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   2 2  2   2    2     2     ', Defense.__name__,"     2     2    2   2  2 2")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 0,3)
                        write_Connect4 (ims)
                        write_uraConnect(ims, Attack, Defense)
                        time.sleep(1)
                        break
                if not win_4o_in(board__1) and not win_4o_in(board__2):
                    rec(total_points, 1, 2, i, i, stime1, stime2)
                    print('\nDRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W')
                    print('  0 0  0   0    0     ', Attack.__name__, " - ", Defense.__name__, "     0    0   0  0 0")
                    print('DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W\n')
                    print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 1,2)
                    write_Connect4 (ims)
                    write_uraConnect(ims, Attack, Defense)
                    time.sleep(3)
                print("\n")
                print(ig,":\n") # , ": ", total_points,
                for itp in total_points.items():
                    print(itp)
                print("\n")
                ig +=1

## bot battle

In [29]:
debugger_c4([
    agent_ConnectX_MarcPaulo, 
    agent_ConnectX_DavidAbelin, 
    agent_ConnectX_Indraneel_Dey, 
    agent_ConnectX_Marco_De_Paolini,
    agent_ConnectX_Kaggle_Learn_AlexisCook]
)



0 :
----------------
('agent_ConnectX_MarcPaulo', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_ConnectX_DavidAbelin', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_ConnectX_Indraneel_Dey', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_ConnectX_Marco_De_Paolini', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_ConnectX_Kaggle_Learn_AlexisCook', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
--------------------------
agent_ConnectX_MarcPaulo   vs   agent_ConnectX_DavidAbelin
-----------------------------------
(0.9) . agent_ConnectX_MarcPaulo > 3  4 < agent_ConnectX_DavidAbelin . (0.3) --- 1 : [3,4] 
(1.0) . agent_ConnectX_MarcPaulo > 3  4 < agent_ConnectX_DavidAbelin . (0.3) --- 2 : [3,4,3,4] 
(1.0) . agent_ConnectX_MarcPaulo > 4  3 < agent_ConnectX_DavidAbelin . (0.3) --- 3 : [3,4,3,4,4,3] 
(1.0) . agent_ConnectX_MarcPaulo > 5  4 < agent_Connect